# 7. Compute Results Overview Tables - Reference

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [33]:
ASSEMBLER_PROCESS_LIST = ["ABYSS", "BCALM2", "GATBMINIAPIPELINE", "METAHIPMER2", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"ABYSS": "AbYSS",
                   "BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "METAHIPMER2": "MetaHipMer2",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}

log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EMS', 'LNN', 'ERR2984773']

genomic_assemblers = ['ABySS', 'BCALM2', 'MINIA', 'SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer']
metagenomic_assemblers = ['MetaHipMer2','GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']
single_kmer = ['BCALM2', 'MINIA', 'ABySS']
multiple_kmer = ['SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer', 'GATBMiniaPipeline', 
                 'IDBA-UD', 'MEGAHIT', 'metaSPAdes', 'MetaHipMer2']

skipped_assemblers = ['ABySS', 'BCALM2', 'MINIA', 'VelvetOptimiser', 'MetaHipMer2']


best_min = ['Ns','misassembled contigs','misassembly events']
best_min_exept_0 = ['contigs', 'L90'] 
best_max = ['LSA', 'NA50','NG50','breadth_of_coverage','identity','lowest_identity']

target_compass = {'multiplicity': 1,'parsimony':1, 'validity':1}
    
target_dict = {
    'Bacillus_subtilis': 4045677,
    'Enterococcus_faecalis': 2845392,
    'Escherichia_coli': 4765434,
    'Lactobacillus_fermentum': 1905333,
    'Listeria_monocytogenes': 2992342,
    'Pseudomonas_aeruginosa': 6792330,
    'Salmonella_enterica': 4759746,
    'Staphylococcus_aureus': 2718780,
}

COLOURS = ['#5876c8', '#58AEC8', '#009392', '#39B185', '#9CCB86', '#E9E29C', '#EEB479', '#E88471', '#CF597E', '#a54765', '#a42a2a', '#835221', 'darkgray']

## Load data

In [21]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        if item['assembler'] not in skipped_assemblers and 'plasmid' not in reference :
                            reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                            'sample': sample,
                                                                                            'assembler': item['assembler'],
                                                                                            'reference': reference,
                                                                                            'LSA': item['contiguity'],
                                                                                            'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                            'multiplicity': item['multiplicity'],
                                                                                            'validity': item['validity'],
                                                                                            'parsimony': item['parsimony'],
                                                                                            'identity': item['identity'],
                                                                                            'lowest_identity': item['lowest_identity'],
                                                                                            'L90': item['L90'],
                                                                                            'contigs': item['aligned_contigs'],
                                                                                            'NA50': item['NA50'],
                                                                                            'NG50': item['NG50'],
                                                                                            'basepairs': item['aligned_basepairs'],
                                                                                            'Ns': item['Ns'],
                                                                                            'misassembled contigs': item['misassembled_contigs'],
                                                                                            'misassembly events': item['misassembly_events']},
                                                                                           ignore_index=True)

reference_pipeline_metrics_df['distribution'] = np.where(reference_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
reference_pipeline_metrics_df = reference_pipeline_metrics_df[reference_pipeline_metrics_df.distribution != 'Log']
reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df['algorythm'] = np.where(reference_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events', 'multiplicity','validity','parsimony','identity','lowest_identity']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events','multiplicity','validity','parsimony','identity','lowest_identity']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...
Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


,L90,LSA,NA50,NG50,Ns,assembler,basepairs,breadth_of_coverage,contigs,identity,...,misassembly events,multiplicity,parsimony,reference,run,sample,validity,distribution,type,algorythm
336,13.0,0.189977,263445.0,263445.0,0.0,GATBMiniaPipeline,3984103.0,0.984780,27.0,0.999864,...,0.0,1.000073,1.000094,Bacillus_subtilis,run1,EMS,0.999979,Even,Metagenomic,Multiple k-mer De Bruijn graph
337,18.0,0.084760,212969.0,212969.0,0.0,IDBA-UD,3977845.0,0.983233,34.0,0.997696,...,0.0,1.000061,1.000084,Bacillus_subtilis,run1,EMS,0.999977,Even,Metagenomic,Multiple k-mer De Bruijn graph
338,13.0,0.189990,263586.0,263586.0,0.0,MEGAHIT,3986471.0,0.985366,27.0,0.999421,...,3.0,0.999952,1.000269,Bacillus_subtilis,run1,EMS,0.999683,Even,Metagenomic,Multiple k-mer De Bruijn graph
339,11.0,0.189944,307964.0,307964.0,0.0,metaSPAdes,3981174.0,0.984056,23.0,0.998251,...,0.0,0.999981,0.999982,Bacillus_subtilis,run1,EMS,0.999998,Even,Metagenomic,Multiple k-mer De Bruijn graph
340,14.0,0.097783,269493.0,263164.0,0.0,SKESA,3971432.0,0.981648,26.0,1.000000,...,0.0,1.000000,1.000000,Bacillus_subtilis,run1,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,19.0,0.152851,126941.0,126941.0,0.0,MEGAHIT,2681132.0,0.986153,37.0,0.998612,...,3.0,0.999880,1.000046,Staphylococcus_aureus,run2,EMS,0.999834,Even,Metagenomic,Multiple k-mer De Bruijn graph
1004,13.0,0.172711,204472.0,204472.0,0.0,metaSPAdes,2682662.0,0.986715,23.0,0.999689,...,0.0,0.999974,0.999978,Staphylococcus_aureus,run2,EMS,0.999997,Even,Metagenomic,Multiple k-mer De Bruijn graph
1005,38.0,0.066829,73147.0,63469.0,0.0,SKESA,2661968.0,0.979104,63.0,1.000000,...,0.0,1.000018,1.000018,Staphylococcus_aureus,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph
1006,11.0,0.186431,280950.0,280950.0,0.0,SPAdes,2678585.0,0.985216,21.0,0.999948,...,0.0,0.999902,0.999902,Staphylococcus_aureus,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph


In [22]:
reference_pipeline_metrics_df.columns

Index(['L90', 'LSA', 'NA50', 'NG50', 'Ns', 'assembler', 'basepairs',
       'breadth_of_coverage', 'contigs', 'identity', 'lowest_identity',
       'misassembled contigs', 'misassembly events', 'multiplicity',
       'parsimony', 'reference', 'run', 'sample', 'validity', 'distribution',
       'type', 'algorythm'],
      dtype='object')

In [23]:
reference_pipeline_metrics_df['sample'].unique()

array(['EMS', 'ERR2984773', 'ENN'], dtype=object)

## Save data


In [24]:
reference_pipeline_metrics_df.to_csv("Tables/Results/Reference metrics_clean.csv")

### Reference stats per assembler - per sample

#### Values

In [25]:
best_stats_dfs = {}

for sample in reference_pipeline_metrics_df['sample'].unique():
    print(sample)
    best_stats_dfs[sample] = {}
    for reference in reference_pipeline_metrics_df['reference'].unique(): 
        print(reference)
        stats_per_assembler = pd.DataFrame()
        best_stats_per_assembler = pd.DataFrame()
        for assembler in reference_pipeline_metrics_df['assembler'].unique():

            row = {'Assembler': assembler, 
                 'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
                 'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}

            best_row = {'Assembler': assembler}

            describe_df = reference_pipeline_metrics_df[(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)].describe()
            for column in describe_df.columns:
                mean = describe_df.loc['mean',column]
                minimum = describe_df.loc['min',column]
                maximum = describe_df.loc['max',column]

                row[column] = "{} [{};{}]".format(round(mean, 2), round(minimum,2), round(maximum,2))

                if "basepairs" in column:
                    target = target_dict[reference]
                    best_row[column] = min(list(reference_pipeline_metrics_df[column][(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)]), key=lambda x:abs(x-target))
                
                elif column in target_compass.keys():
                    target=1
                    best_row[column] = min(list(reference_pipeline_metrics_df[column][(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)]), key=lambda x:abs(x-target))
                elif column in best_min_exept_0:
                    temp = [x for x in list(reference_pipeline_metrics_df[column][(reference_pipeline_metrics_df['reference'] == reference) & (reference_pipeline_metrics_df['assembler'] == assembler) & (reference_pipeline_metrics_df['sample'] == sample)]) if x != 0]
                    if len(temp) > 0:
                        best_row[column] = min(temp)
                    else:
                        best_row[column] = 0
                elif column in best_min:
                    best_row[column] = minimum
                elif column in best_max:
                    best_row[column] = maximum

            stats_per_assembler=stats_per_assembler.append(row, ignore_index=True)
            best_stats_per_assembler=best_stats_per_assembler.append(best_row, ignore_index=True)

        stats_per_assembler = stats_per_assembler.set_index(['Assembler','Algorithm', 'Type'])
        best_stats_per_assembler = best_stats_per_assembler.set_index('Assembler')
        display(stats_per_assembler)
        display(best_stats_per_assembler)
        stats_per_assembler.to_csv("Tables/Results/Reference metrics per assembler - {} {}.csv".format(sample, reference))
        best_stats_per_assembler.to_csv("Tables/Results/Best Reference metrics per assembler - {} {}.csv".format(sample, reference))
        best_stats_dfs[sample][reference] = best_stats_per_assembler

EMS
Bacillus_subtilis


,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.19 [0.19;0.19],263445.0 [263445.0;263445.0],263445.0 [263445.0;263445.0],0.0 [0.0;0.0],3984103.0 [3984103.0;3984103.0],0.98 [0.98;0.98],27.0 [27.0;27.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,18.0 [18.0;18.0],0.08 [0.08;0.08],212969.0 [212969.0;212969.0],212969.0 [212969.0;212969.0],0.0 [0.0;0.0],3977845.0 [3977845.0;3977845.0],0.98 [0.98;0.98],34.0 [34.0;34.0],1.0 [1.0;1.0],0.92 [0.92;0.92],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.19 [0.19;0.19],263586.0 [263586.0;263586.0],263586.0 [263586.0;263586.0],0.0 [0.0;0.0],3986471.0 [3986471.0;3986471.0],0.99 [0.99;0.99],27.0 [27.0;27.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],3.0 [3.0;3.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,11.0 [11.0;11.0],0.19 [0.19;0.19],307964.0 [307964.0;307964.0],307964.0 [307964.0;307964.0],0.0 [0.0;0.0],3981174.0 [3981174.0;3981174.0],0.98 [0.98;0.98],23.0 [23.0;23.0],1.0 [1.0;1.0],0.97 [0.97;0.97],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,14.0 [14.0;14.0],0.1 [0.1;0.1],269493.0 [269493.0;269493.0],263164.0 [263164.0;263164.0],0.0 [0.0;0.0],3971432.0 [3971432.0;3971432.0],0.98 [0.98;0.98],26.0 [26.0;26.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SPAdes,Multiple k-mer De Bruijn graph,Genomic,11.0 [11.0;11.0],0.19 [0.19;0.19],315090.0 [315090.0;315090.0],315090.0 [315090.0;315090.0],0.0 [0.0;0.0],3981330.0 [3981330.0;3981330.0],0.98 [0.98;0.98],22.0 [22.0;22.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
Unicycler,Multiple k-mer De Bruijn graph,Genomic,11.0 [11.0;11.0],0.19 [0.19;0.19],314937.0 [314937.0;314937.0],314937.0 [314937.0;314937.0],0.0 [0.0;0.0],3975910.0 [3975910.0;3975910.0],0.98 [0.98;0.98],20.0 [20.0;20.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,13.0,0.189977,263445.0,263445.0,0.0,3984103.0,0.984780,27.0,0.999864,0.996848,0.0,0.0,1.000073,1.000094,0.999979
IDBA-UD,18.0,0.084760,212969.0,212969.0,0.0,3977845.0,0.983233,34.0,0.997696,0.922166,0.0,0.0,1.000061,1.000084,0.999977
MEGAHIT,13.0,0.189990,263586.0,263586.0,0.0,3986471.0,0.985366,27.0,0.999421,0.995636,1.0,3.0,0.999952,1.000269,0.999683
metaSPAdes,11.0,0.189944,307964.0,307964.0,0.0,3981174.0,0.984056,23.0,0.998251,0.971025,0.0,0.0,0.999981,0.999982,0.999998
SKESA,14.0,0.097783,269493.0,263164.0,0.0,3971432.0,0.981648,26.0,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000
SPAdes,11.0,0.190053,315090.0,315090.0,0.0,3981330.0,0.984095,22.0,0.999995,0.999911,0.0,0.0,0.999922,0.999923,0.999999
Unicycler,11.0,0.189918,314937.0,314937.0,0.0,3975910.0,0.982755,20.0,0.999981,0.999612,0.0,0.0,1.000000,1.000000,1.000000


Enterococcus_faecalis


,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.19 [0.19;0.19],235254.0 [235254.0;235254.0],235254.0 [235254.0;235254.0],0.0 [0.0;0.0],2807690.33 [2784031.0;2819520.0],0.99 [0.98;0.99],28.67 [28.0;29.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,28.0 [28.0;28.0],0.14 [0.14;0.14],113206.0 [113206.0;113206.0],113206.0 [113206.0;113206.0],0.0 [0.0;0.0],2811602.0 [2811602.0;2811602.0],0.99 [0.99;0.99],53.0 [53.0;53.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,13.0 [13.0;13.0],0.23 [0.23;0.23],235316.0 [235316.0;235316.0],238546.0 [238546.0;238546.0],0.0 [0.0;0.0],2810735.33 [2786540.0;2822833.0],0.99 [0.98;0.99],31.67 [31.0;32.0],1.0 [1.0;1.0],0.95 [0.95;0.95],1.0 [1.0;1.0],2.0 [2.0;2.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,11.0 [11.0;11.0],0.19 [0.19;0.19],238611.0 [238611.0;238611.0],238611.0 [238611.0;238611.0],0.0 [0.0;0.0],2786440.0 [2786440.0;2786440.0],0.98 [0.98;0.98],23.0 [23.0;23.0],1.0 [1.0;1.0],0.98 [0.98;0.98],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,19.0 [19.0;19.0],0.14 [0.14;0.14],152709.0 [152709.0;152709.0],152709.0 [152709.0;152709.0],0.0 [0.0;0.0],2810284.0 [2810284.0;2810284.0],0.99 [0.99;0.99],40.0 [40.0;40.0],0.98 [0.98;0.98],0.37 [0.37;0.37],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.02 [1.02;1.02],0.98 [0.98;0.98]
SPAdes,Multiple k-mer De Bruijn graph,Genomic,14.0 [14.0;14.0],0.19 [0.19;0.19],235662.0 [235662.0;235662.0],215153.0 [215153.0;215153.0],0.0 [0.0;0.0],2781588.0 [2781588.0;2781588.0],0.98 [0.98;0.98],26.0 [26.0;26.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
Unicycler,Multiple k-mer De Bruijn graph,Genomic,8.0 [8.0;8.0],0.23 [0.23;0.23],377104.0 [377104.0;377104.0],377104.0 [377104.0;377104.0],0.0 [0.0;0.0],2783327.0 [2783327.0;2783327.0],0.98 [0.98;0.98],20.0 [20.0;20.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]


...

,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,96.0 [96.0;96.0],0.06 [0.06;0.06],52959.0 [52959.0;52959.0],51487.0 [51487.0;51487.0],0.0 [0.0;0.0],4643726.0 [4643659.0;4643859.0],0.98 [0.98;0.98],178.0 [178.0;178.0],1.0 [1.0;1.0],0.99 [0.99;0.99],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,171.67 [171.0;172.0],0.04 [0.04;0.04],26251.0 [25757.0;26498.0],27717.0 [27717.0;27717.0],0.0 [0.0;0.0],4551054.0 [4550745.0;4551672.0],0.96 [0.96;0.96],396.33 [396.0;397.0],0.99 [0.99;0.99],0.66 [0.66;0.66],7.33 [7.0;8.0],12.67 [12.0;14.0],1.01 [1.01;1.01],1.02 [1.02;1.02],0.99 [0.99;0.99]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,47.0 [47.0;47.0],0.08 [0.08;0.08],117581.0 [117581.0;117581.0],117581.0 [117581.0;117581.0],0.0 [0.0;0.0],4669364.0 [4669364.0;4669364.0],0.98 [0.98;0.98],103.0 [103.0;103.0],1.0 [1.0;1.0],0.93 [0.93;0.93],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,25.0 [25.0;25.0],0.11 [0.11;0.11],198784.0 [198784.0;198784.0],198784.0 [198784.0;198784.0],0.0 [0.0;0.0],4674406.0 [4674406.0;4674406.0],0.98 [0.98;0.98],56.0 [56.0;56.0],1.0 [1.0;1.0],0.93 [0.93;0.93],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,144.0 [144.0;144.0],0.03 [0.03;0.03],37980.0 [37980.0;37980.0],37661.0 [37661.0;37661.0],0.0 [0.0;0.0],4566475.0 [4566475.0;4566475.0],0.96 [0.96;0.96],245.0 [245.0;245.0],1.0 [1.0;1.0],0.22 [0.22;0.22],2.0 [2.0;2.0],2.0 [2.0;2.0],1.0 [1.0;1.0],1.01 [1.01;1.01],0.99 [0.99;0.99]
SPAdes,Multiple k-mer De Bruijn graph,Genomic,46.0 [46.0;46.0],0.11 [0.11;0.11],103636.0 [103636.0;103636.0],103102.0 [103102.0;103102.0],0.0 [0.0;0.0],4614007.0 [4614007.0;4614007.0],0.97 [0.97;0.97],91.0 [91.0;91.0],1.0 [1.0;1.0],0.99 [0.99;0.99],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
Unicycler,Multiple k-mer De Bruijn graph,Genomic,35.0 [35.0;35.0],0.08 [0.08;0.08],157802.0 [157802.0;157802.0],155040.0 [155040.0;155040.0],0.0 [0.0;0.0],4637757.0 [4637757.0;4637757.0],0.97 [0.97;0.97],89.0 [89.0;89.0],1.0 [1.0;1.0],0.99 [0.99;0.99],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,96.0,0.057981,52959.0,51487.0,0.0,4643859.0,0.975653,178.0,0.999949,0.991722,0.0,0.0,1.001513,1.001514,0.999999
IDBA-UD,171.0,0.037402,26498.0,27717.0,0.0,4551672.0,0.956285,396.0,0.994826,0.655810,7.0,12.0,1.007920,1.015506,0.992530
MEGAHIT,47.0,0.079200,117581.0,117581.0,0.0,4669364.0,0.981011,103.0,0.999158,0.929582,0.0,0.0,1.001243,1.001308,0.999935
metaSPAdes,25.0,0.111373,198784.0,198784.0,0.0,4674406.0,0.982070,56.0,0.998099,0.934869,0.0,0.0,1.000126,1.000165,0.999960
SKESA,144.0,0.031122,37980.0,37661.0,0.0,4566475.0,0.959395,245.0,0.996346,0.221785,2.0,2.0,1.001434,1.009822,0.991694
SPAdes,46.0,0.111353,103636.0,103102.0,0.0,4614007.0,0.969381,91.0,0.999293,0.987168,0.0,0.0,1.000159,1.000161,0.999998
Unicycler,35.0,0.077132,157802.0,155040.0,0.0,4637757.0,0.974371,89.0,0.999713,0.991144,0.0,0.0,1.000000,1.000000,1.000000


Staphylococcus_aureus


,,,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,Algorithm,Type,,,,,,,,,,,,,,,
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,39.0 [39.0;39.0],0.07 [0.07;0.07],63844.0 [63844.0;63844.0],53404.0 [53404.0;53404.0],0.0 [0.0;0.0],2670675.0 [2670675.0;2670675.0],0.98 [0.98;0.98],65.0 [65.0;65.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,43.0 [43.0;43.0],0.07 [0.07;0.07],53248.0 [53248.0;53248.0],53248.0 [53248.0;53248.0],0.0 [0.0;0.0],2665812.0 [2665812.0;2665812.0],0.98 [0.98;0.98],74.0 [74.0;74.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,14.0 [14.0;14.0],0.13 [0.13;0.13],181811.0 [181811.0;181811.0],213210.0 [213210.0;213210.0],0.0 [0.0;0.0],2681291.0 [2681291.0;2681291.0],0.99 [0.99;0.99],40.0 [40.0;40.0],1.0 [1.0;1.0],0.99 [0.99;0.99],3.0 [3.0;3.0],7.0 [7.0;7.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,8.0 [8.0;8.0],0.38 [0.38;0.38],360908.0 [360908.0;360908.0],360908.0 [360908.0;360908.0],0.0 [0.0;0.0],2681445.0 [2681445.0;2681445.0],0.99 [0.99;0.99],19.0 [19.0;19.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SKESA,Multiple k-mer De Bruijn graph,Genomic,43.0 [43.0;43.0],0.06 [0.06;0.06],53048.0 [53048.0;53048.0],53048.0 [53048.0;53048.0],0.0 [0.0;0.0],2652026.0 [2652026.0;2652026.0],0.98 [0.98;0.98],73.0 [73.0;73.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
SPAdes,Multiple k-mer De Bruijn graph,Genomic,14.0 [14.0;14.0],0.21 [0.21;0.21],214432.0 [214432.0;214432.0],214432.0 [214432.0;214432.0],0.0 [0.0;0.0],2670466.0 [2670466.0;2670466.0],0.98 [0.98;0.98],24.0 [24.0;24.0],1.0 [1.0;1.0],0.93 [0.93;0.93],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]
Unicycler,Multiple k-mer De Bruijn graph,Genomic,14.0 [14.0;14.0],0.31 [0.31;0.31],223263.0 [223263.0;223263.0],164016.0 [164016.0;164016.0],0.0 [0.0;0.0],2670223.0 [2670223.0;2670223.0],0.98 [0.98;0.98],25.0 [25.0;25.0],1.0 [1.0;1.0],1.0 [1.0;1.0],0.0 [0.0;0.0],0.0 [0.0;0.0],1.0 [1.0;1.0],1.0 [1.0;1.0],1.0 [1.0;1.0]


,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,39.0,0.066903,63844.0,53404.0,0.0,2670675.0,0.982306,65.0,1.000000,1.000000,0.0,0.0,1.000670,1.000670,1.000000
IDBA-UD,43.0,0.066902,53248.0,53248.0,0.0,2665812.0,0.980518,74.0,0.999978,0.998542,0.0,0.0,1.000006,1.000006,1.000000
MEGAHIT,14.0,0.133521,181811.0,213210.0,0.0,2681291.0,0.986211,40.0,0.999509,0.992685,3.0,7.0,0.999851,1.000377,0.999474
metaSPAdes,8.0,0.381399,360908.0,360908.0,0.0,2681445.0,0.986268,19.0,0.999732,0.996403,0.0,0.0,1.000026,1.000026,1.000000
SKESA,43.0,0.064607,53048.0,53048.0,0.0,2652026.0,0.975447,73.0,1.000000,0.999979,0.0,0.0,1.000018,1.000018,1.000000
SPAdes,14.0,0.211306,214432.0,214432.0,0.0,2670466.0,0.982230,24.0,0.996927,0.934990,0.0,0.0,0.999987,1.000014,0.999973
Unicycler,14.0,0.309299,223263.0,164016.0,0.0,2670223.0,0.982140,25.0,0.999991,0.999788,0.0,0.0,0.999966,0.999966,1.000000


#### rank

In [30]:
rank_dfs = {}
for sample in best_stats_dfs:
    print(sample)
    rank_dfs[sample] = {}
    for reference in best_stats_dfs[sample]:
        df = best_stats_dfs[sample][reference]
        
        rank_df = pd.DataFrame(index=df.index)

        for column in df.columns:
            rank_list = []

            if "basepairs" in column:
                target = target_dict[reference]
                for item in df[column]:
                    diff = 1 - abs(item-target) / target
                    rank_list.append(diff)
                rank_df[column] = rank_list
            elif column in target_compass:
                target = 1
                for item in df[column]:
                    diff = 1- abs(item-target) / target
                    rank_list.append(diff)
                rank_df[column] = rank_list
            else:    
                if column in best_min:
                    for item in df[column]:
                        diff = 1 - (item / df[column].max())
                        rank_list.append(diff)
                else:
                    for item in df[column]:
                        diff = (item / df[column].max())
                        rank_list.append(diff)

                rank_df[column] = rank_list

        # in case assembly fails
        for assembler, row in df.iterrows():
            if row.sum() == 0:
                rank_df.at[assembler, :]=0

        rank_df = rank_df.fillna(1) #happens when all values are 0
        display(rank_df) 
        rank_df.to_csv("Tables/Results/Rank Global metrics per assembler - {}.csv".format(sample))
        rank_dfs[sample][reference]= rank_df


EMS


/tmp/ipykernel_6703/3757550408.py:28: RuntimeWarning:

invalid value encountered in double_scalars



,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.722222,0.999599,0.836094,0.836094,1.0,0.984780,0.999406,0.794118,0.999864,0.996848,1.0,1.0,0.999927,0.999906,0.999979
IDBA-UD,1.000000,0.445983,0.675899,0.675899,1.0,0.983233,0.997836,1.000000,0.997696,0.922166,1.0,1.0,0.999939,0.999916,0.999977
MEGAHIT,0.722222,0.999671,0.836542,0.836542,1.0,0.985366,1.000000,0.794118,0.999421,0.995636,0.0,0.0,0.999952,0.999731,0.999683
metaSPAdes,0.611111,0.999425,0.977384,0.977384,1.0,0.984056,0.998671,0.676471,0.998251,0.971025,1.0,1.0,0.999981,0.999982,0.999998
SKESA,0.777778,0.514503,0.855289,0.835203,1.0,0.981648,0.996227,0.764706,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000
SPAdes,0.611111,1.000000,1.000000,1.000000,1.0,0.984095,0.998710,0.647059,0.999995,0.999911,1.0,1.0,0.999922,0.999923,0.999999
Unicycler,0.611111,0.999289,0.999514,0.999514,1.0,0.982755,0.997351,0.588235,0.999981,0.999612,1.0,1.0,1.000000,1.000000,1.000000


/tmp/ipykernel_6703/3757550408.py:28: RuntimeWarning:

invalid value encountered in double_scalars



,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.464286,0.806869,0.623844,0.623844,1.0,0.990907,0.998826,0.528302,0.999991,0.999719,1.0,1.0,0.999617,0.999617,1.000000
IDBA-UD,1.000000,0.578086,0.300198,0.300198,1.0,0.988125,0.996021,1.000000,0.999975,0.998788,1.0,1.0,0.999772,0.999771,0.999999
MEGAHIT,0.464286,1.000000,0.624008,0.632574,1.0,0.992072,1.000000,0.584906,0.998077,0.950710,0.0,0.0,0.999977,0.999979,0.999956
metaSPAdes,0.392857,0.807076,0.632746,0.632746,1.0,0.979282,0.987108,0.433962,0.998336,0.980807,1.0,1.0,0.999838,0.999838,1.000000
SKESA,0.678571,0.577890,0.404952,0.404952,1.0,0.987661,0.995554,0.754717,0.984162,0.366465,0.0,0.5,1.000000,0.976641,0.977174
SPAdes,0.500000,0.806776,0.624926,0.570540,1.0,0.977576,0.985389,0.490566,0.999934,0.998737,1.0,1.0,0.999823,0.999821,0.999998
Unicycler,0.285714,0.999711,1.000000,1.000000,1.0,0.978188,0.986005,0.377358,1.000000,1.000000,1.0,1.0,0.999841,0.999841,1.000000


...

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.116418,0.314815,0.344260,0.344260,1.0,0.987016,0.996118,0.177921,0.999491,0.957871,0.571429,0.600000,0.999158,0.998692,0.999534
IDBA-UD,0.105970,0.379058,0.338466,0.338466,1.0,0.985660,0.994750,0.172562,0.998623,0.862126,0.714286,0.733333,0.999308,0.999171,0.999864
MEGAHIT,0.043284,0.522165,0.947888,0.947888,1.0,0.990862,1.000000,0.087889,0.999837,0.989578,1.000000,1.000000,0.999591,0.999591,0.999999
metaSPAdes,0.035821,1.000000,1.000000,1.000000,1.0,0.990271,0.999403,0.060021,0.999518,0.982412,1.000000,1.000000,0.999883,0.999883,1.000000
SKESA,1.000000,0.048096,0.039221,0.038380,1.0,0.963658,0.972545,1.000000,0.998831,0.250874,0.000000,0.000000,0.999949,0.996893,0.996851
SPAdes,0.038806,0.947181,0.882696,0.882696,1.0,0.988865,0.997984,0.071811,0.999538,0.990849,0.714286,0.800000,0.999973,0.999974,0.999999
Unicycler,0.037313,0.944280,0.994583,0.927118,1.0,0.987023,0.996125,0.082529,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


/tmp/ipykernel_6703/3757550408.py:28: RuntimeWarning:

invalid value encountered in double_scalars



,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.561404,0.520606,0.266415,0.259010,1.0,0.975653,0.993465,0.449495,1.000000,1.000000,1.000000,1.000000,0.998487,0.998486,0.999999
IDBA-UD,1.000000,0.335826,0.133300,0.139433,1.0,0.956285,0.973743,1.000000,0.994877,0.661284,0.000000,0.000000,0.992080,0.984494,0.992530
MEGAHIT,0.274854,0.711129,0.591501,0.591501,1.0,0.981011,0.998921,0.260101,0.999209,0.937342,1.000000,1.000000,0.998757,0.998692,0.999935
metaSPAdes,0.146199,1.000000,1.000000,1.000000,1.0,0.982070,1.000000,0.141414,0.998149,0.942673,1.000000,1.000000,0.999874,0.999835,0.999960
SKESA,0.842105,0.279439,0.191062,0.189457,1.0,0.959395,0.976910,0.618687,0.996397,0.223636,0.714286,0.833333,0.998566,0.990178,0.991694
SPAdes,0.269006,0.999828,0.521350,0.518663,1.0,0.969381,0.987079,0.229798,0.999344,0.995408,1.000000,1.000000,0.999841,0.999839,0.999998
Unicycler,0.204678,0.692563,0.793837,0.779942,1.0,0.974371,0.992160,0.224747,0.999764,0.999418,1.000000,1.000000,1.000000,1.000000,1.000000


/tmp/ipykernel_6703/3757550408.py:28: RuntimeWarning:

invalid value encountered in double_scalars



,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,
GATBMiniaPipeline,0.906977,0.175415,0.176898,0.147971,1.0,0.982306,0.995984,0.878378,1.000000,1.000000,1.0,1.0,0.999330,0.999330,1.000000
IDBA-UD,1.000000,0.175413,0.147539,0.147539,1.0,0.980518,0.994170,1.000000,0.999978,0.998542,1.0,1.0,0.999994,0.999994,1.000000
MEGAHIT,0.325581,0.350081,0.503760,0.590760,1.0,0.986211,0.999943,0.540541,0.999509,0.992685,0.0,0.0,0.999851,0.999623,0.999474
metaSPAdes,0.186047,1.000000,1.000000,1.000000,1.0,0.986268,1.000000,0.256757,0.999732,0.996403,1.0,1.0,0.999974,0.999974,1.000000
SKESA,1.000000,0.169394,0.146985,0.146985,1.0,0.975447,0.989029,0.986486,1.000000,0.999979,1.0,1.0,0.999982,0.999982,1.000000
SPAdes,0.325581,0.554028,0.594146,0.594146,1.0,0.982230,0.995906,0.324324,0.996927,0.934990,1.0,1.0,0.999987,0.999986,0.999973
Unicycler,0.325581,0.810958,0.618615,0.454454,1.0,0.982140,0.995815,0.337838,0.999991,0.999788,1.0,1.0,0.999966,0.999966,1.000000


### radar plots

In [100]:
for sample in rank_dfs:
    print(sample)
    fig = make_subplots(rows=2, cols=4, specs=[[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}],[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]], subplot_titles= ['<i>' + x.replace('_',' ') + '</i><br>' for x in sorted(rank_dfs[sample])])
    col_n=1
    row_n=1
    for reference in sorted(rank_dfs[sample]):
        print(reference)
        print(col_n,row_n)
        showlegend=True if col_n==1 and row_n==1 else False

        df = rank_dfs[sample][reference]
        i= 0

        for assembler, row in df.iterrows():

            row_data = []

            for col in df.columns:
                row_data.append(row.loc[:].at[col])

            fig.add_trace(go.Scatterpolar(r=row_data,
                                          theta=list([z.replace('_',' ') for z in df.columns]), mode='lines+markers',
                                          marker=dict(color=COLOURS[i], size=12), 
                                          marker_line_color="black", 
                                          marker_line_width=2,
                                          opacity=0.6,
                                          name=assembler, line=dict(color=COLOURS[i]), showlegend=showlegend),
                          col=col_n, row=row_n)

            i+=1
        if row_n == 2:
            row_n = 1
            col_n += 1
        else:
            row_n += 1
    
    fig.update_layout(polar=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar2=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar3=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar4=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar5=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar6=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar7=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5),
                     polar8=dict(radialaxis=dict(visible=True,
                                                 range=[0,1],
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                radialaxis_angle = -22.5))
    
    
    fig.update_layout(title=dict(text='<b>'+sample,
                                 font=dict(size=42, color='black'),
                                 y=0.97,
                                 x=0.5,
                                 xanchor='center',
                                 yanchor='top'),
                     font=dict(size=14))
    fig.layout.annotations[0].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[1].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[2].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[3].update(y=0.54, font=dict(size=20,color="black"))
    fig.layout.annotations[4].update(y=0.42, font=dict(size=20,color="black"))
    fig.layout.annotations[5].update(y=0.42, font=dict(size=20,color="black"))
    fig.layout.annotations[6].update(y=0.42, font=dict(size=20,color="black"))
    fig.layout.annotations[7].update(y=0.42, font=dict(size=20,color="black"))

    fig.show()
    plot(fig, filename='Plots/Reference Metrics/Radial plot - {}.html'.format(sample), auto_open=False)



EMS
Bacillus_subtilis
1 1
Enterococcus_faecalis
1 2
Escherichia_coli
2 1
Lactobacillus_fermentum
2 2
Listeria_monocytogenes
3 1
Pseudomonas_aeruginosa
3 2
Salmonella_enterica
4 1
Staphylococcus_aureus
4 2


ERR2984773
Bacillus_subtilis
1 1
Enterococcus_faecalis
1 2
Escherichia_coli
2 1
Lactobacillus_fermentum
2 2
Listeria_monocytogenes
3 1
Pseudomonas_aeruginosa
3 2
Salmonella_enterica
4 1
Staphylococcus_aureus
4 2


ENN
Bacillus_subtilis
1 1
Enterococcus_faecalis
1 2
Escherichia_coli
2 1
Lactobacillus_fermentum
2 2
Listeria_monocytogenes
3 1
Pseudomonas_aeruginosa
3 2
Salmonella_enterica
4 1
Staphylococcus_aureus
4 2


In [29]:
for sample in rank_dfs:
    print(sample)
    for reference in rank_dfs[sample]:
        print(reference)
        fig = go.Figure()
        df = rank_dfs[sample][reference]
        for assembler in df.index:

            fig.add_trace(go.Box(x=df.loc[[assembler]].values[0],
                                     name=assembler, boxpoints='all',jitter=1, pointpos=0,
                                     boxmean=False, fillcolor='#D3D3D3', line=dict(color='#000000'),
                                    marker=dict(color='rgba(178,37,34,0.7)', size=6)))
        fig.update_layout(showlegend=False, xaxis_title="Score",
                              plot_bgcolor='rgb(255,255,255)', xaxis=dict(zeroline=False, gridcolor='#DCDCDC'),
                         title=sample)
        fig.show()


EMS
Bacillus_subtilis


Enterococcus_faecalis


Escherichia_coli


Lactobacillus_fermentum


Listeria_monocytogenes


...

Lactobacillus_fermentum


Listeria_monocytogenes


Pseudomonas_aeruginosa


Salmonella_enterica


Staphylococcus_aureus
